In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

## YF Financial Scraper
You can also use this template to scrape balance sheet and cashflow statement, just have to enable the url accordingly.

In [2]:
path = "chromedriver.exe"
service = webdriver.chrome.service.Service(path)
service.start()
options = Options()
options.add_argument("--headless") 

ticker = "AAPL" # Choose any ticker you want the financial data of. Here we are using Apple.
url = "https://finance.yahoo.com/quote/{}/financials?p={}".format(ticker,ticker)
#url = "https://finance.yahoo.com/quote/{}/balance-sheet?p={}".format(ticker,ticker)
#url = "https://finance.yahoo.com/quote/{}/cash-flow?p={}".format(ticker,ticker)

driver = webdriver.Chrome(service=service, options = options)
driver.get(url)
driver.implicitly_wait(3)

### clicking dropdown buttons before scraping   
buttons = driver.find_elements(By.XPATH,  '//section[@class="main svelte-e2c64s"]//button')
for button in buttons:
    if button.accessible_name in ["","Follow","Quarterly","Annual","prev","next"]:
        pass
    else:
        WebDriverWait(driver, 1).until(EC.element_to_be_clickable(button)).click()

table = driver.find_element(By.XPATH,  '//div[@class="tableContainer svelte-1pgoo1f"]')
text_blob = table.text.split("\n")
income_st_dir = {}
last_key = None
for count, row in enumerate(text_blob):
    if count == 0:
        heading = row.split()
        column_count = len(heading[1:])
    else:
        if count%(column_count+1) == 1:
            income_st_dir[row] = []
            last_key = row
        else:
            income_st_dir[last_key].append(row)        
df = pd.DataFrame(income_st_dir).T
df.columns = heading[1:]

for col in df.columns:
    df[col] = df[col].str.replace(r'[,\|s-]', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

driver.close()

In [3]:
df

,TTM,9/30/2023,9/30/2022,9/30/2021,9/30/2020
Total Revenue,3.816230e+08,3.832850e+08,3.943280e+08,3.658170e+08,2.745150e+08
Cost of Revenue,2.076570e+08,2.141370e+08,2.235460e+08,2.129810e+08,1.695590e+08
Gross Profit,1.739660e+08,1.691480e+08,1.707820e+08,1.528360e+08,1.049560e+08
Operating Expense,5.572600e+07,5.484700e+07,5.134500e+07,4.388700e+07,3.866800e+07
Selling General and Administrative,2.537800e+07,2.493200e+07,2.509400e+07,2.197300e+07,1.991600e+07
Research & Development,3.034800e+07,2.991500e+07,2.625100e+07,2.191400e+07,1.875200e+07
Operating Income,1.182400e+08,1.143010e+08,1.194370e+08,1.089490e+08,6.628800e+07
Net Non Operating Interest Income Expense,NaN,1.830000e+05,1.060000e+05,1.980000e+05,8.900000e+05
Interest Income Non Operating,NaN,3.750000e+06,2.825000e+06,2.843000e+06,3.763000e+06
Interest Expense Non Operating,NaN,3.933000e+06,2.931000e+06,2.645000e+06,2.873000e+06


## Key Statistics Scraper

In [6]:
ticker = "AAPL" # Choose any ticker you want the statistics data of. Here we are using Apple.
url = 'https://finance.yahoo.com/quote/{}/key-statistics'.format(ticker)

path = "chromedriver.exe"
service = webdriver.chrome.service.Service(path)
service.start()
options = Options()
options.add_argument("--headless") 

driver = webdriver.Chrome(service=service, options = options)
driver.get(url)
driver.implicitly_wait(0.2)

table = driver.find_element(By.XPATH,  '//div[@class="table-container svelte-104jbnt"]')
text_blob = table.text.split("\n")
financial_metrics = {}
header = text_blob[0].split()

for item in text_blob[1:]:
    parts = item.split()
    key_end = len(parts) - len(header)
    key = ' '.join(parts[:key_end])
    values = parts[key_end:]
    financial_metrics[key] = dict(zip(header, values))

df = pd.DataFrame(financial_metrics).T
df = df.replace({'T': 'E+12', 'B': 'E+09', 'M': 'E+06', 'K': 'E+03'}, regex=True)
df = df.apply(pd.to_numeric, errors='coerce')

In [7]:
df

,Current,3/31/2024,12/31/2023,9/30/2023,6/30/2023,3/31/2023
Market Cap,2.920000e+12,2.650000e+12,2.990000e+12,2.680000e+12,3.050000e+12,2.610000e+12
Enterprise Value,2.950000e+12,2.680000e+12,3.040000e+12,2.720000e+12,3.100000e+12,2.670000e+12
Trailing P/E,2.955000e+01,2.667000e+01,3.141000e+01,2.873000e+01,3.288000e+01,2.800000e+01
Forward P/E,2.899000e+01,2.632000e+01,2.915000e+01,2.577000e+01,2.941000e+01,2.786000e+01
PEG Ratio (5yr expected),2.250000e+00,2.110000e+00,2.310000e+00,2.180000e+00,2.660000e+00,2.790000e+00
Price/Sales,7.780000e+00,6.990000e+00,7.940000e+00,7.100000e+00,8.080000e+00,6.890000e+00
Price/Book,3.926000e+01,3.549000e+01,4.790000e+01,4.417000e+01,4.908000e+01,4.599000e+01
Enterprise Value/Revenue,7.730000e+00,6.960000e+00,7.940000e+00,7.090000e+00,8.060000e+00,6.890000e+00
Enterprise Value/EBITDA,2.219000e+01,2.010000e+01,2.356000e+01,2.146000e+01,2.451000e+01,2.084000e+01
